<a href="https://colab.research.google.com/github/BLayus/Data_Folia/blob/main/Bloco_de_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Bloco de Dados**

#Precisamos escolher a trilha sonora do nosso carnaval! Utilizando o banco de dados music-store.db, vamos tentar encontrar as melhores músicas para os nossos convidados.

In [ ]:
# Usando SQLITE3

import sqlite3
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Create a SQL connection to our SQLite database
con = sqlite3.connect('/content/drive/MyDrive/music-store.db')
cur = con.cursor()

# The result of a "cursor.execute" can be iterated over by row
for row in cur.execute("SELECT * FROM sqlite_master WHERE type = 'table'"):
  print(row[1:])

('sqlite_sequence', 'sqlite_sequence', 3, 'CREATE TABLE sqlite_sequence(name,seq)')
('artists', 'artists', 2, 'CREATE TABLE artists\n(\n    id INTEGER PRIMARY KEY AUTOINCREMENT,\n    name VARCHAR(120)\n)')
('albums', 'albums', 4, 'CREATE TABLE albums\n(\n    id INTEGER PRIMARY KEY AUTOINCREMENT,\n    title VARCHAR(160)  NOT NULL,\n    artist_id INTEGER  NOT NULL,\n    FOREIGN KEY (artist_id) REFERENCES artists (id)\n    ON DELETE NO ACTION ON UPDATE NO ACTION\n)')
('employees', 'employees', 5, 'CREATE TABLE employees\n(\n    id INTEGER PRIMARY KEY AUTOINCREMENT,\n    last_name VARCHAR(20)  NOT NULL,\n    first_name VARCHAR(20)  NOT NULL,\n    title VARCHAR(30),\n    reports_to INTEGER,\n    birth_date TIMESTAMP,\n    hire_date TIMESTAMP,\n    address VARCHAR(70),\n    city VARCHAR(40),\n    state VARCHAR(40),\n    country VARCHAR(40),\n    postal_code VARCHAR(10),\n    phone VARCHAR(24),\n    fax VARCHAR(24),\n    email VARCHAR(60),\n    FOREIGN KEY (reports_to) REFERENCES employees (i

In [ ]:
def query(text):
  
  con = sqlite3.connect('/content/drive/MyDrive/music-store.db')
  df = pd.read_sql(text, con)
  con.close()
  return df

In [ ]:
schema = query('select * from sqlite_master where type = "table"')
schema

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,artists,artists,2,CREATE TABLE artists\n(\n id INTEGER PRIMAR...
2,table,albums,albums,4,CREATE TABLE albums\n(\n id INTEGER PRIMARY...
3,table,employees,employees,5,CREATE TABLE employees\n(\n id INTEGER PRIM...
4,table,customers,customers,6,CREATE TABLE customers\n(\n id INTEGER PRIM...
5,table,genres,genres,7,CREATE TABLE genres\n(\n id INTEGER PRIMARY...
6,table,invoices,invoices,8,CREATE TABLE invoices\n(\n id INTEGER PRIMA...
7,table,media_types,media_types,9,CREATE TABLE media_types\n(\n id INTEGER PR...
8,table,tracks,tracks,10,CREATE TABLE tracks\n(\n id INTEGER PRIMARY...
9,table,invoice_lines,invoice_lines,11,CREATE TABLE invoice_lines\n(\n id INTEGER ...


In [ ]:
def show_schema(tabela):
  
  for i in (schema.loc[schema['tbl_name'] == tabela, 'sql']):
      print(i)

In [ ]:
show_schema('tracks')

CREATE TABLE tracks
(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name VARCHAR(200)  NOT NULL,
    album_id INTEGER,
    media_type_id INTEGER  NOT NULL,
    genre_id INTEGER,
    composer VARCHAR(220),
    milliseconds INTEGER  NOT NULL,
    bytes INTEGER,
    unit_price NUMERIC(10,2)  NOT NULL,
    FOREIGN KEY (album_id) REFERENCES albums (id)
    ON DELETE NO ACTION ON UPDATE NO ACTION,
    FOREIGN KEY (genre_id) REFERENCES genres (id)
    ON DELETE NO ACTION ON UPDATE NO ACTION,
    FOREIGN KEY (media_type_id) REFERENCES media_types (id)
    ON DELETE NO ACTION ON UPDATE NO ACTION
)


**Qual artista possui mais albuns?**

In [ ]:
data = query('''SELECT artists.id, artists.name, count(title) total_albums
              FROM artists
              INNER JOIN albums ON artists.id = albums.artist_id
              GROUP BY name
              ORDER BY total_albums DESC''' )
data.head()

,id,name,total_albums
0,90,Iron Maiden,21
1,22,Led Zeppelin,14
2,58,Deep Purple,11
3,50,Metallica,10
4,150,U2,10


**Existem clientes de quantos países?**

In [ ]:
data = query('SELECT count(DISTINCT country) FROM customers')
data

,count(DISTINCT country)
0,24


**Quantos clientes são brasileiros?**

In [ ]:
data = query('select count(id) from customers where country="Brazil"')
data

,count(id)
0,5


**Qual genero possui mais músicas?**

In [ ]:
data = query('''select count(tracks.name) as track_qtd, 
            genres.name as genero
            from genres 
            inner join tracks on genres.id = tracks.genre_id 
            group by genero
            order by track_qtd DESC''')
data

,track_qtd,genero
0,1297,Rock
1,579,Latin
2,374,Metal
3,332,Alternative & Punk
4,130,Jazz
5,93,TV Shows
6,81,Blues
7,74,Classical
8,64,Drama
9,61,R&B/Soul


**Qual genero possui mais albuns?**

In [ ]:
data = query('''select count(distinct tracks.album_id) as albums_qtd, 
            genres.name as genero
            from genres 
            inner join tracks on genres.id = tracks.genre_id 
            group by genero
            order by albums_qtd DESC''')
data

,albums_qtd,genero
0,117,Rock
1,72,Classical
2,39,Latin
3,35,Metal
4,23,Alternative & Punk
5,13,Jazz
6,10,TV Shows
7,7,Blues
8,5,Alternative
9,4,Drama


**Exiba os 5 clientes que mais fizeram pedidos.**

In [ ]:
query ("""
SELECT customer_id, COUNT (*) as total FROM invoices 
LEFT JOIN customers as Cust ON invoices.customer_id = cust.id
GROUP BY customer_id LIMIT 5

""")

,customer_id,total
0,1,7
1,2,7
2,3,7
3,4,7
4,5,7


In [ ]:
show_schema('customers')
show_schema('invoices')
show_schema('invoice_lines')

CREATE TABLE customers
(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name VARCHAR(40)  NOT NULL,
    last_name VARCHAR(20)  NOT NULL,
    company VARCHAR(80),
    address VARCHAR(70),
    city VARCHAR(40),
    state VARCHAR(40),
    country VARCHAR(40),
    postal_code VARCHAR(10),
    phone VARCHAR(24),
    fax VARCHAR(24),
    email VARCHAR(60)  NOT NULL,
    support_rep_id INTEGER,
    FOREIGN KEY (support_rep_id) REFERENCES employees (id)
    ON DELETE NO ACTION ON UPDATE NO ACTION
)
CREATE TABLE invoices
(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id INTEGER  NOT NULL,
    invoice_date TIMESTAMP  NOT NULL,
    billing_address VARCHAR(70),
    billing_city VARCHAR(40),
    billing_state VARCHAR(40),
    billing_country VARCHAR(40),
    billing_postal_code VARCHAR(10),
    total NUMERIC(10,2)  NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES customers (id)
    ON DELETE NO ACTION ON UPDATE NO ACTION
)
CREATE TABLE invoice_lines
(
    id INTEGER PRIMARY KEY AUT

In [ ]:
data = query('''SELECT first_name, last_name, count(quantity) as total_pedidos
              FROM customers as cust INNER JOIN invoices as inv ON cust.id = inv.customer_id INNER JOIN invoice_lines as invl on inv.id = invl.invoice_id
              GROUP BY cust.id
              ORDER BY total_pedidos DESC ''')
data

,first_name,last_name,total_pedidos
0,Luís,Gonçalves,38
1,Leonie,Köhler,38
2,François,Tremblay,38
3,Bjørn,Hansen,38
4,František,Wichterlová,38
5,Helena,Holý,38
6,Astrid,Gruber,38
7,Daan,Peeters,38
8,Kara,Nielsen,38
9,Eduardo,Martins,38


**Qual é o valor da maior compra?**

**Quais o email dos 10 clientes que mais gastaram na loja?**

**Qual o valor médio de cada compra?**

In [ ]:
query("""
SELECT AVG(total) AS total_medio FROM invoices
""")

,total_medio
0,5.651942


**Quantas compras estão acima da média?**

In [ ]:
query("""
SELECT * FROM invoices WHERE total > (SELECT AVG(total) AS total_medio FROM invoices)

""")

,id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
0,3,8,2007-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
1,4,14,2007-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
2,5,23,2007-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86
3,10,46,2007-02-03 00:00:00,3 Chatham Street,Dublin,Dublin,Ireland,None,5.94
4,11,52,2007-02-06 00:00:00,202 Hoxton Street,London,None,United Kingdom,N1 5LH,8.91
...,...,...,...,...,...,...,...,...,...
174,403,56,2011-11-08 00:00:00,307 Macacha Güemes,Buenos Aires,None,Argentina,1106,8.91
175,404,6,2011-11-13 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,25.86
176,409,29,2011-12-06 00:00:00,796 Dundas Street West,Toronto,ON,Canada,M6J 1V1,5.94
177,410,35,2011-12-09 00:00:00,"Rua dos Campeões Europeus de Viena, 4350",Porto,None,Portugal,None,8.91


**Vamos fazer uma playlist com as 12 músicas mais vendidas! Quais foram elas? Quais os generos e os artistas?**

In [ ]:
query("""
SELECT SUM (invoices.quantity)
        FROM invoice_lines
        GROUP BY track_id
        """)

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 8))



TypeError: ignored